# Temperature Anomalies Calculation Notebook.


In [1]:
#get things
import numpy as np
import pandas as pd 
from netCDF4 import Dataset 
import numpy.ma as ma 

filename1 = "CERES_SYN1deg-Month_200003-201509_Basic.nc"
ds1 = Dataset(filename1, mode="r")


In [9]:
print(ds1) # checking the data set variables, dimensions, sizes, etc so I can use that info later

<type 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    title: CERES SYN1deg Products - Monthly Means
    institution: NASA/LaRC (Langley Research Center) Hampton, Va
    Conventions: CF-1.4
    comment: Data is from East to West and South to North. See values in latitude and longitudes dimensions.
    version: This is version 3A: May 2, 2014
    Fill_Value: Fill Value is -999.0
    dimensions(sizes): lon(360), lat(180), time(187)
    variables(dimensions): float32 lon(lon), float32 lat(lat), int32 time(time), float32 toa_net_all_mon(time,lat,lon), float32 toa_net_clr_mon(time,lat,lon), float32 aux_skint_mon(time,lat,lon)
    groups: 



In [2]:
#pull out relevant data
time = ds1.variables['time'][0:180]
lat = ds1.variables['lat'][:]
lon = ds1.variables['lon'][:]
toa = ds1.variables['toa_net_all_mon'][0:180,:,:]
skin = ds1.variables['aux_skint_mon'][0:180,:,:]

# this part I was just checking the attributes for each variable so I could save it all the same again later. 
print(np.shape(time)) 

(180L,)


In [11]:
np.max(skin)

319.23517

In [12]:
np.min(skin)

194.3334

In [13]:
np.max(toa)

196.20041

In [14]:
np.min(toa)

-214.69356

# Calculating anomalies


## Calculating monthly averages & monthly deviations

Here we are taking into account the natural variation between months. The cell below just gathers all the indices corresponding to each month over all available (complete) years.

In [3]:
#pull out which indices would correspond to the same month in our dataset
months=np.array([0,1,2,3,4,5,6,7,8,9,10,11])
all_months=np.array([0,1,2,3,4,5,6,7,8,9,10,11])

for i in range(1,15): #each month occurs 15 times except for two months extra
    months+=12
    all_months=np.vstack((all_months, months))

all_months = np.transpose(all_months) #so that it is divided properly

print(all_months) #check that it works, just missing the last two months which I add back in later

[[  0  12  24  36  48  60  72  84  96 108 120 132 144 156 168]
 [  1  13  25  37  49  61  73  85  97 109 121 133 145 157 169]
 [  2  14  26  38  50  62  74  86  98 110 122 134 146 158 170]
 [  3  15  27  39  51  63  75  87  99 111 123 135 147 159 171]
 [  4  16  28  40  52  64  76  88 100 112 124 136 148 160 172]
 [  5  17  29  41  53  65  77  89 101 113 125 137 149 161 173]
 [  6  18  30  42  54  66  78  90 102 114 126 138 150 162 174]
 [  7  19  31  43  55  67  79  91 103 115 127 139 151 163 175]
 [  8  20  32  44  56  68  80  92 104 116 128 140 152 164 176]
 [  9  21  33  45  57  69  81  93 105 117 129 141 153 165 177]
 [ 10  22  34  46  58  70  82  94 106 118 130 142 154 166 178]
 [ 11  23  35  47  59  71  83  95 107 119 131 143 155 167 179]]


In [7]:
#initialize variables in same shape as toa/skin arrays. 
toa_loc_monthly_anomaly = np.empty([180,180,360])
skin_loc_monthly_anomaly = np.empty([180,180,360])


#calculate monthly averages and deviations from those averages
for i in range(1,13):
    submonths = all_months[i-1] #all others were just 15 years exactly
    
    #actual calculation for each month, stored in the same places as original data
    toa_loc_monthly_anomaly[submonths,:,:] = toa[submonths,:,:] - np.mean(toa[submonths,:,:],0)
    skin_loc_monthly_anomaly[submonths,:,:] = skin[submonths,:,:] - np.mean(skin[submonths,:,:],0)
    

In [6]:
np.shape(toa_loc_monthly_anomaly)

(180L, 180L, 360L)

### Annual Deviations from Average


In [6]:
#calculate the average over time at each lat/lon position
toaAvgLL = np.mean(toa[:,:,:],0)
skinAvgLL = np.mean(skin[:,:,:],0)
all_years = np.transpose(all_months)

In [5]:
skin_av = np.empty([180,180,360])
toa_av = np.empty([180,180,360])

for i in range(1,16):
    skin_av[all_years[i-1],:,:] = np.mean(skin[all_years[i-1],:,:],0)-skinAvgLL
    toa_av[all_years[i-1],:,:] = np.mean(toa[all_years[i-1],:,:],0)-toaAvgLL
    
 

In [9]:
np.min(skin_av)

-10.5908203125

In [10]:
np.max(skin_av)

12.37420654296875

## Saving the new data as a netCDF

This section takes our newly calculated variables and saves them into a netCDF format. If you calculate more variables, just add them using the format of a variable it is similar too. Just be sure to use a unique name. This section will not be necessary when we convert it to a visit script since we will just directly plot the data.

In [9]:
# Save new data as new netCDF file

#sources
#http://salishsea-meopar-tools.readthedocs.org/en/latest/netcdf4/
#http://nbviewer.jupyter.org/urls/bitbucket.org/salishsea/tools/raw/tip/I_ForcingFiles/Initial/PrepareTS.ipynb
#http://stackoverflow.com/questions/28462521/saving-climatology-result-to-netcdf-file


anom = Dataset('anomalies_b5.nc','w')
anom.createDimension('time', 180)
anom.createDimension('lat', 180)
anom.createDimension('lon', 360)

anom_lat = anom.createVariable('lat', 'float32', ('lat'), zlib=True)
anom_lat.long_name='Latitude'
anom_lat.units = 'degrees_north'
anom_lat[:]= lat

anom_lon = anom.createVariable('lon', 'float32', ('lon'), zlib=True)
anom_lon.long_name='Longitude'
anom_lon.units = 'degrees_east'
anom_lon[:]= lon

anom_time = anom.createVariable('time', 'int32', ('time'), zlib=True)
anom_time.units = 'days since 2000-03-01 00:00:00'
anom_time.longname = 'time'
anom_time[:] = time

anom_toag = anom.createVariable('toa_av', 'float32', ('time','lat','lon'), zlib=True, fill_value=-999.0)
anom_toag.units = 'W m-2'
anom_toag.longname = 'Annual TOA Net flux subtract all time average'
anom_toag.coordinates = 'time lat lon'
anom_toag.valid_range= (-400,400)
anom_toag[:] = toa_av

anom_skg = anom.createVariable('skin_av', 'float32', ('time','lat','lon'), zlib=True, fill_value=-999.0)
anom_skg.units = 'K'
anom_skg.longname = 'Annual Skin Temperature subtract all time average'
anom_skg.coordinates = 'time lat lon'
anom_skg.valid_range= (-400,400)
anom_skg[:] = skin_av

anom.close()